In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np
import matplotlib.pyplot as plt #Package for visualization
import re #importing package for Regular expression operations
from sklearn.model_selection import train_test_split #Package for splitting the data
from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical
from keras.preprocessing.text import Tokenizer #Tokenization
from keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils import to_categorical

In [2]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('Sentiment.csv')

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

In [17]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-17-cee1da567eb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-17-cee1da567eb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [4]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [5]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix


In [6]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [7]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [8]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split

In [9]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 51s - loss: 0.8223 - accuracy: 0.6493 - 51s/epoch - 175ms/step
144/144 - 2s - loss: 0.7555 - accuracy: 0.6796 - 2s/epoch - 11ms/step
0.75551837682724
0.6795544028282166


In [10]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


#1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [11]:
model.save('sentimentAnalysis.h5') #Saving the model

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [13]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [14]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 250ms/epoch - 250ms/step
[0.6810064  0.11271847 0.20627514]
Neutral


#2. Apply GridSearchCV on the source code provided in the class

In [15]:
pip install scikeras

In [16]:
from scikeras.wrappers import KerasClassifier #importing Keras classifier

from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(model=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

744/744 - 113s - loss: 0.8229 - accuracy: 0.6454 - 113s/epoch - 152ms/step
186/186 - 2s - 2s/epoch - 12ms/step


744/744 - 114s - loss: 0.8204 - accuracy: 0.6499 - 114s/epoch - 153ms/step
186/186 - 2s - 2s/epoch - 10ms/step


744/744 - 102s - loss: 0.8246 - accuracy: 0.6439 - 102s/epoch - 136ms/step
186/186 - 2s - 2s/epoch - 10ms/step


744/744 - 104s - loss: 0.8218 - accuracy: 0.6472 - 104s/epoch - 139ms/step
186/186 - 3s - 3s/epoch - 14ms/step


744/744 - 101s - loss: 0.8203 - accuracy: 0.6473 - 101s/epoch - 136ms/step
186/186 - 2s - 2s/epoch - 10ms/step


...

Epoch 1/2
186/186 - 34s - loss: 0.8451 - accuracy: 0.6388 - 34s/epoch - 183ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6893 - accuracy: 0.7061 - 24s/epoch - 131ms/step
47/47 - 1s - 621ms/epoch - 13ms/step


Epoch 1/2
186/186 - 32s - loss: 0.8523 - accuracy: 0.6312 - 32s/epoch - 173ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6835 - accuracy: 0.7086 - 24s/epoch - 128ms/step
47/47 - 1s - 634ms/epoch - 13ms/step


Epoch 1/2
186/186 - 33s - loss: 0.8492 - accuracy: 0.6348 - 33s/epoch - 177ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6868 - accuracy: 0.6994 - 24s/epoch - 130ms/step
47/47 - 1s - 603ms/epoch - 13ms/step


Epoch 1/2
186/186 - 33s - loss: 0.8461 - accuracy: 0.6371 - 33s/epoch - 176ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6786 - accuracy: 0.7102 - 24s/epoch - 127ms/step
47/47 - 1s - 588ms/epoch - 13ms/step


Epoch 1/2
233/233 - 37s - loss: 0.8307 - accuracy: 0.6451 - 37s/epoch - 158ms/step
Epoch 2/2
233/233 - 30s - loss: 0.6809 - accuracy: 0.7091 - 30s/epoch - 129ms/step
Best: 0.680404 using {'batch_size': 40, 'epochs': 2}
